In [1]:
import pandas as pd
import os
import psycopg2
import csv
from scipy import stats

Here we will find the file names where the raw data lives (inside /data folder).

### To create new database from pgloader:
run these commands in the terminal:

    1.  brew install pgloader         # possibly already installed
    2.  createdb uw_mad_grades        # createdb {databasename}  
    3.  pgloader database.sqlite3 postgresql:///uw_mad_grades       # pgloader {sqllite3 filename} postgresql:///{database name}
    
We will be using a combination of PostgreSQL and also pandas.read_csv(...) since the data is available in both forms.
When we only need 1 table at a time, pandas DataFrame will be more convenient. Let's walk through the data folder and read
the files, saving them each to their own variable.

In [2]:
file_names = []
for (dirpath, dirnames, filenames) in os.walk('data'):
    file_names.extend(filenames)
    break
    
file_names = list(filter(lambda x:'csv' in x, filenames))
file_names

['subjects.csv',
 'subject_memberships.csv',
 'rooms.csv',
 'schedules.csv',
 'sections.csv',
 'courses.csv',
 'course_offerings.csv',
 'instructors.csv',
 'teachings.csv',
 'grade_distributions.csv']

In [62]:
tables = {}
for table in file_names:
    with open('data/' + table) as f:
        tables[table[:-4]] = pd.read_csv(f)
        
subjects = tables['subjects']
subject_memberships = tables['subject_memberships']
rooms = tables['rooms']
schedules = tables['schedules']
sections = tables['sections']
courses = tables['courses']
course_offerings = tables['course_offerings']
instructors = tables['instructors']
teachings = tables['teachings']
grade_distributions = tables['grade_distributions']
        
print(f'converted into {len(tables)} dataframes stored in the dictionary called "tables" as well as their individual filenames')

converted into 10 dataframes stored in the dictionary called "tables" as well as their individual filenames


Now we begin pulling data from Postgres, first we create a "helper-table" to cast a few important columns as numbers instead of strings.

In [7]:
conn = psycopg2.connect("dbname=uw_mad_grades")
cur = conn.cursor()
query = '''
DROP TABLE IF EXISTS grade_dist_numeric
'''
cur.execute(query)
query = '''
CREATE TABLE grade_dist_numeric AS (
  SELECT
    course_offering_uuid
  , section_number
  , CAST(grade_distributions.a_count AS integer)
  , CAST(grade_distributions.ab_count AS integer)
  , CAST(grade_distributions.b_count AS integer)
  , CAST(grade_distributions.bc_count AS integer)
  , CAST(grade_distributions.c_count AS integer)
  , CAST(grade_distributions.d_count AS integer)
  , CAST(grade_distributions.f_count AS integer)
  FROM grade_distributions
  )
'''
cur.execute(query)
conn.commit()

This is the big query that will get us all the data we need for the rest of the project. 
Protip: Formatting a query this large is much easier in a code editor such as Vim, since it will color code variable names and do other helpful things, whereas Jupyter will treat everything inside the string as raw text, which is harder to format nicely.

In [8]:
conn = psycopg2.connect("dbname=uw_mad_grades")
cur = conn.cursor()

query = '''
SELECT
  sections.course_offering_uuid
, course_offerings.course_uuid
, course_offerings.name
, COUNT(sections.number) AS section_count
, instructor_id
, SUM(a_count) AS a_count
, SUM(ab_count) AS ab_count
, SUM(b_count) AS b_count
, SUM(bc_count) AS bc_count
, SUM(c_count) AS c_count
, SUM(d_count) AS d_count
, SUM(f_count) AS f_count
FROM
   teachings
INNER JOIN
   sections
   ON teachings.section_uuid=sections.uuid
INNER JOIN
   grade_dist_numeric
   ON (
     (sections.course_offering_uuid = grade_dist_numeric.course_offering_uuid)
      AND (sections.number = grade_dist_numeric.section_number))
INNER JOIN
   course_offerings
   ON sections.course_offering_uuid=course_offerings.uuid
WHERE
   (a_count + ab_count + b_count + bc_count + c_count + d_count + f_count) > 0
GROUP BY
  sections.course_offering_uuid
, course_offerings.course_uuid
, course_offerings.name
, instructor_id
HAVING
  count(instructor_id) > 1;
'''

cur.execute(query)
all_grades_count_df = pd.DataFrame(cur.fetchall())
all_grades_count_df.columns = [i[0] for i in cur.description]

In [65]:
all_grades_count_df.head(2)

,course_offering_uuid,course_uuid,name,section_count,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,class_size,counts
0,000d1ef4-b78a-304e-8ff9-382dfc96835a,10362c9c-bbf8-39e3-8506-c2783ee31625,Diversity in Special Education,3,3802457,96,15,0,0,0,0,0,111,5
1,00193853-8f6d-3e14-994b-77a557b35d74,a4dcc153-1a04-39d8-9b28-b56fb4fa5266,Cult Anthro&Human Diversity,2,3409966,368,272,109,42,21,8,6,826,20


In [96]:
col = ['a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count']
all_grades_count_df['class_size'] = all_grades_count_df[col].sum(axis=1)
all_grades_count_df['counts'] = all_grades_count_df.groupby(['course_uuid'])['course_uuid'].transform('count')
all_grades_count_df["pct_A"] = all_grades_count_df['a_count'] / all_grades_count_df['class_size']
all_grades_count_df['pct_F'] = all_grades_count_df['f_count'] / all_grades_count_df['class_size']

In [98]:
#Which class was taught the most times?
all_grades_count_df = all_grades_count_df.loc[all_grades_count_df['counts']>1]
all_grades_count_df.sort_values('counts', ascending=False).head(1)

,course_offering_uuid,course_uuid,name,section_count,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,class_size,counts,pct_A,pct_F
5794,78a11bde-275c-313a-9c65-ea93faacdeba,7a552326-da08-354c-aafc-f56edcf83f44,Intro to Speech Composition,2,5516801,5,14,2,0,0,0,1,22,489,0.227273,0.045455


Cool, looks like "Intro to Speech Composition" was taught a total of 489 times. Let's suppose we are advising the Head of the Communication Arts Dept. at UW-Madison, who would like to know whether any professors in the department are "outliers" in terms of grades given. ie. Does anyone give A's "too freely" or alternatively do they give F's too readily? We will find list of professors from [the official website](https://commarts.wisc.edu/people?rid=All&field_person_affiliation_tid=11) and compare their percentage of a chosen grade (A or F) with the overall percentage of grades given for anyone who has taught the class.

In [130]:
instructors

,id,name
0,761703,JOHN ARCHAMBAULT
1,3677061,STEPHANIE KANN
2,788586,KATHY PREM
3,1600463,KRISTIN KLARKOWSKI
4,693634,DAVID BOHNHOFF
5,590599,RICHARD STRAUB
6,664097,KEVIN SHINNERS
7,2600979,SUNDARAM GUNASEKARAN
8,4847746,ROBERT ANEX
9,3213425,KRISHNAPURAM KARTHIKEYAN


In [145]:
current_profs = '''Paul Ahn, Joanne Cantor, Tammy Chang, Anthony Chen, Larisa Doroshenko, Mary Anne Fitzpatrick, Xinle Jia, Sangwon Lee, 
                    Yang Liu, Marie-Louise Mares, AnneMarie McClain, David Mortensen, Julian Mueller-Herbst, Esther Paik, Zhongdang Pan, 
                    Alanna Peebles, Irene Sarmiento, Maura Snyder, Catalina L. Toma, Fangjing Tu, Lyn Van Swol, Michael Xenos'''

current_profs = current_profs.split(',')
current_profs = [name.strip().upper() for name in current_profs]
#print(current_profs)

current_profs_df = instructors.head(0)
display(current_profs_df)
for name in current_profs:
    temp = instructors.loc[instructors['name'] == name]
    if len(temp) > 0:
        display(temp)
        current_profs_df.append(temp)

# current_profs_id = []
# for prof in current_profs:
#     if prof.upper() in instructors['name']:
#         print('winner', prof)
#         current_profs_id.append(prof)
# current_profs_id

,id,name


,id,name
8890,4909736,LARISA DOROSHENKO


,id,name
8964,5697124,XINLE JIA


,id,name
8893,5186059,SANGWON LEE


,id,name
1051,4145440,YANG LIU
2797,5334549,YANG LIU
8846,4715210,YANG LIU


...

,id,name
258,5519194,IRENE SARMIENTO


,id,name
8955,5863072,MAURA SNYDER


,id,name
8898,5181640,FANGJING TU


,id,name
12503,4384631,LYN VAN SWOL


,id,name
8894,1591621,MICHAEL XENOS


In [99]:
prof_matters = all_grades_count_df.loc[all_grades_count_df['course_uuid'] == '7a552326-da08-354c-aafc-f56edcf83f44'].drop(['course_offering_uuid', 'course_uuid', 'section_count', 'name', 'counts'], axis=1)
std_of_pct_A = prof_matters['pct_A'].std()
prof_matters.head(0)

,instructor_id,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,class_size,pct_A,pct_F


In [84]:
def prof_a_size(df, grade='a'):
    '''Input a dataframe with grade counts and the professors who gave those grades, along with the 
       exact grade letter of interest. Output two dictionaries, each with professor as the keys, 
       ( {prof : df} , {prof : [%<grade> of THIS prof, %<grade> of OTHER profs]})'''
    profs = df['instructor_id'].values
    profs_dict = {}
    profs_pct = {}
    grade_search = grade.lower() + '_count'
    for prof in profs:
        profs_dict[prof] = df.loc[df['instructor_id'] == prof, [grade_search, 'class_size']]   #str(grade_search)
        other_profs = df.loc[df['instructor_id'] != prof, [grade_search, 'class_size']]
        pct_other = other_profs.sum()[0] / other_profs.sum()[1]
        
        pct_a = profs_dict[prof].sum()[0] / profs_dict[prof].sum()[1]
        profs_pct[prof] = pct_a, pct_other
        
    return profs_dict, profs_pct

profs_dict, profs_pct = prof_a_size(prof_matters, grade='a')

In [164]:
profs_pct['3196989'][0]
prof_matters['pct_A'].std()

0.16534193035249733

In [167]:
def prof_hypothesis_test(instructor_id, alpha=.05):
    
    instructor_pct = profs_pct[instructor_id][0]
    stdev = prof_matters['pct_A'].std()
    n = len(profs_dict[instructor_id])

    # p +- z* . E
    #min<p<max - True otherwise False
    if n == 1:
        lower = instructor_pct - stdev
        upper = instructor_pct + stdev
    else:
        lower = instructor_pct + stats.t.ppf(alpha / 2, n-1) * stdev / n ** .5
        upper = instructor_pct - stats.t.ppf(alpha / 2, n-1) * stdev / n ** .5
    #print(n, 'samples gives:', lower, ',', upper)
    if(lower < instructor_pct) and (instructor_pct < upper):
        return True
    print(n, 'samples gives:', lower, ',', upper)
    return False

for prof in prof_matters['instructor_id'].values:
    if prof_hypothesis_test(prof) == False:
        print(prof, 'is outside of normal')

In [91]:
len(profs_dict['4709223'])

6

In [69]:
print("Easiest professors : (their % A's , other prof % A's)")
display([(key, val) for (key, val) in sorted(profs_pct.items(), key=lambda x: x[1][0], reverse=True)][:5])
print('----------------------------------')
print("Hardest professors : (their % A's , other prof % A's)")
display([(key, val) for (key, val) in sorted(profs_pct.items(), key=lambda x: x[1][0], reverse=True)][-5:])

Easiest professors : (their % A's , other prof % A's)


[('3960315', (0.8, 0.23397720843772732)),
 ('4402193', (0.7551020408163265, 0.23305530812211514)),
 ('4619996', (0.7345132743362832, 0.23052503052503054)),
 ('4398942', (0.6923076923076923, 0.2341577756223731)),
 ('5905783', (0.6666666666666666, 0.2342815581057055))]

----------------------------------
Hardest professors : (their % A's , other prof % A's)


[('3776752', (0.04081632653061224, 0.2358895457122034)),
 ('4602472', (0.04, 0.23551281015113554)),
 ('3719062', (0.04, 0.23551281015113554)),
 ('3639549', (0.038461538461538464, 0.23553184610410605)),
 ('1625191', (0.03125, 0.23617642289605967))]

In [72]:
def extreme_profs(n=3, profs_dict=profs_dict, profs_pct=profs_pct):
    i=0
    diff = []
    
    profs_pct_sorted = [(key, val) for (key, val) in sorted(profs_pct.items(), key=lambda x: x[1][0], reverse=True)]
    for prof, pct_tuple in profs_pct_sorted:
        print(f"{prof} gives {round(pct[0] * 100, 1)}% , while all other profs give {round(pct[1] * 100, 1)}%")
        diff.append([prof, pct[0] - pct[1]])
    
    for prof, df in profs_dict.items():
        display(df)
        i += 1
        if i>n:
            break

In [64]:
diff = []
diff.append([prof, pct[0] - pct[1]])
diff[:10]

[['5340443', -0.005829487490891426]]

In [60]:
display(teachings.head(2))
sections.head(2)

,instructor_id,section_uuid
0,761703,45adf63c-48c9-3659-8561-07556d2d4ddf
1,761703,c6280e23-5e43-3859-893e-540d94993529


,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
1,c6280e23-5e43-3859-893e-540d94993529,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,FLD,1,NaN,f2d66a4d-0c08-3b48-abf6-649fffd7ae90


Now let's test whether the GPA for large classes is different from small classes.

In [63]:
big_classes = all_grades_count_df.loc[all_grades_count_df['class_size'] > 100]
small_classes = all_grades_count_df.loc[all_grades_count_df['class_size'] < 25]